In [1]:
import os
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/pranay/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package

[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to /home/pranay/nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentiwordnet to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package sentiwordnet is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package sentence_polarity is already up-to-date!
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/pranay/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    

ModuleNotFoundError: No module named 'sklearn.cross_validation'

In [2]:
flares = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]


rindi = pd.read_csv('rindiaeda.csv')
rindi.head()

,title,url,comments,flair,score,id,author,body
0,Coronavirus (COVID-19) Megathread - News and U...,https://www.reddit.com/r/india/comments/g1zi21...,1802.0,Coronavirus,138.0,g1zi21,IAmMohit,###[Covid-19 Fundraisers & Donation Links](htt...
1,"[Monthly Happiness Thread] Randians, please sh...",https://www.reddit.com/r/india/comments/g4d2ix...,52.0,Scheduled,32.0,g4d2ix,cool_boyy,<3 \n \nLinks: ...
2,RIP Prof. G. D. Agarwal. A selfless eco-warrio...,https://www.reddit.com/r/india/comments/g4ppv1...,18.0,Non-Political,444.0,g4ppv1,be_yourself_2020,**TL;DR : IITian-turned-activist Professor GD ...
3,[showerthoughts] People finding old demonetize...,https://www.reddit.com/r/india/comments/g4tv6h...,8.0,Non-Political,66.0,g4tv6h,longpostshitpost,Had they cleaned up their places at least once...
4,Three Hindu men in Karnataka held for causing ...,https://www.reddit.com/r/india/comments/g4nhgb...,9.0,Politics,164.0,g4nhgb,silentr3b31,https://www.businessinsider.in/india/news/thre...


In [3]:

def get_date(created):
    return dt.datetime.fromtimestamp(created)

def string_form(value):
    return str(value)

def clean_text(text):
     from sklearn.linear_model import SGDClassifier
     text = BeautifulSoup(text, "lxml").text
     text = text.lower()
     text = REPLACE_BY_SPACE_RE.sub(' ', text)
     text = BAD_SYMBOLS_RE.sub('', text)
     text = ' '.join(word for word in text.split() if word not in STOPWORDS)
     return text


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))



In [5]:
rindi['title'] = rindi['title'].apply(string_form)
rindi['body'] = rindi['body'].apply(string_form)
rindi['comments'] = rindi['comments'].apply(string_form)

rindi['title'] = rindi['title'].apply(clean_text)
rindi['body'] = rindi['body'].apply(clean_text)
rindi['comments'] = rindi['comments'].apply(clean_text)

feature_combine = rindi["title"] + rindi["comments"] + rindi["url"] + rindi['body']
rindi = rindi.assign(feature_combine = feature_combine)

/home/pranay/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


In [6]:
def nb_classifier(X_train, X_test, y_train, y_test):
  
  from sklearn.naive_bayes import MultinomialNB


  nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
  nb.fit(X_train, y_train)

  y_pred = nb.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flares))


In [7]:
def linear_svm(X_train, X_test, y_train, y_test):
  
  from sklearn.linear_model import SGDClassifier

  sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                 ])
  sgd.fit(X_train, y_train)

  y_pred = sgd.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))

In [8]:

def logisticreg(X_train, X_test, y_train, y_test):

    from sklearn.linear_model import LogisticRegression

    logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e30)),
                 ])
    logreg.fit(X_train, y_train)

    y_pred = logreg.predict(X_test)

    print('accuracy %s' % accuracy_score(y_pred, y_test))
   # print(classification_report(y_test, y_pred,target_names=flairs))


In [9]:

def randomforest(X_train, X_test, y_train, y_test):
  
  from sklearn.ensemble import RandomForestClassifier
  
  ranfor = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 42)),
                 ])
  ranfor.fit(X_train, y_train)

  y_pred = ranfor.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))


In [10]:

def mlpclassifier(X_train, X_test, y_train, y_test):
  
  from sklearn.neural_network import MLPClassifier
  
  mlp = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MLPClassifier(hidden_layer_sizes=(30,30,30))),
                 ])
  mlp.fit(X_train, y_train)

  y_pred = mlp.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  #print(classification_report(y_test, y_pred,target_names=flairs))

In [11]:
def train_test(X,y):
 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 42)

 # print("Results of Naive Bayes Classifier")
  #nb_classifier(X_train, X_test, y_train, y_test)
  #print("Results of Linear Support Vector Machine")
  #linear_svm(X_train, X_test, y_train, y_test)
  #print("Results of Logistic Regression")
  #logisticreg(X_train, X_test, y_train, y_test)
  print("Results of Random Forest")
  randomforest(X_train, X_test, y_train, y_test)
  #print("Results of MLP Classifier")
  #mlpclassifier(X_train, X_test, y_train, y_test)

In [12]:
cat = rindi.flair

V = rindi.feature_combine
W = rindi.comments
X = rindi.title
Y = rindi.body
Z = rindi.url

print("Flair Detection using Title as Feature:")
print("\n")
train_test(X,cat)
print("\n")
print("Flair Detection using Body as Feature:")
print("\n")
train_test(Y,cat)
print("\n")
print("Flair Detection using URL as Feature:")
print("\n")
train_test(Z,cat)
print("\n")
print("Flair Detection using Comments as Feature:")
print("\n")
train_test(W,cat)
print("\n")
print("Flair Detection using combined features as Feature:")
print("\n")
train_test(V,cat)


Flair Detection using Title as Feature:


Results of Random Forest
accuracy 0.794392523364486


Flair Detection using Body as Feature:


Results of Random Forest
accuracy 0.7850467289719626


Flair Detection using URL as Feature:


Results of Random Forest
accuracy 0.5794392523364486


Flair Detection using Comments as Feature:


Results of Random Forest
accuracy 0.514018691588785


Flair Detection using combined features as Feature:


Results of Random Forest
accuracy 0.7819314641744548


In [13]:
import pickle

In [15]:
pickle.dump(train_test, open('model.sav','wb'))